# Primal 3.0
Redone from the second section of Prof. Kruczenski's Maple script file `K_sp_leaf_points.maple`, while accounting for $b$.

Remarks:
- At first glance, it looks like the data points generately using `scipy` alone are not **as accurate as** Maple, but seem to be **accurate enough** (hopefully).
- Implementing `sympy` will grant us the ability to manipulate the variables/data to an arbitrary precision, like Maple and Mathematica.
- In the case where the data generated using Python is pretty far off the more precise values generated in Maple/Mathematica, we will use Python solely for importing the data files and optimizing with `cvxpy`, without generating data in Python.

In [1]:
import os
import numpy as np
from scipy.special import lqn

# import scipy as sp
# import sympy as smp
# import cvxpy as cp
# import matplotlib.pyplot as plt
# import mpmath as mp

## Numerical Implementation
For the primal problem, we have to compute the primal functional and the partial waves to impose unitarity. For that purpose, we map the region $s \in [4, \infty)$ to $\xi \in [0, \pi]$ using
$$
s = \frac{4}{\cos^2 \left( \frac{\xi}{2} \right)}.
$$

Through a redefinition of the constant $f_0$ and the spectral density $\sigma(x)$, we can rewrite the amplitude $F(s, t, u)$ in a neater way, visualized later, where $F$ depends on the kernel $\mathcal{K}$, given by
$$
\mathcal{K}(\xi, s) = \frac{1}{\pi} \frac{\sin(\xi)}{\frac{8}{s} - 1 - \cos(\xi)}.
$$

To compute the partial waves, it is useful to define
$$
\begin{align*}
    \hat{\mathbf{\Phi}}_{\ell}(s; \xi_{1}) & = \int_{-1}^{+1} P_{\ell}(\mu) \mathcal{K}(\xi, t) ~ \mathrm{d} \mu, \\
    \tilde{\mathbf{\Phi}}_{\ell}(s; \xi_{1}, \xi_{2}) & = \int_{-1}^{+1} P_{\ell}(\mu) \mathcal{K}(\xi, t) \mathcal{K}(\xi, u) ~ \mathrm{d} \mu, \\
\end{align*}
$$
where, as usual, $t$ and $u$ are function of $\mu$.

### Interpolation Points

We will discretize $\xi$ by choosing equally space points, such that
$$
\xi_j = j \Delta_{\xi}, \quad \Delta_{\xi} = \frac{\pi}{M}, \quad 0 \leq j \leq M, \quad s_j = \frac{4}{\cos^2(\frac{\xi_j}{2})}.
$$

Specifically, the grid will be divided into $2M$ pieces, where $\xi_j$ will take on values in the range $[0, 2 \pi)$ in segments of $\frac{\pi}{M}$.

The primal functional becomes
$$
\mathcal{F}_P = a f_0 + a_{j_1} \sigma_{j_1} + a_{j_1, j_2} \rho_{j_1, j_2},
$$
where the coefficients are
$$
\begin{align*}
    a & = 1, \\
    a_{j_1} & = \Delta_{\xi} \left( \mathcal{K}(\xi_{j_1}, s_0) + \mathcal{K}(\xi_{j_1}, t_0) + \mathcal{K}(\xi_{j_1}, u_0) \right), \\
    a_{j_1, j_2} & = \frac{\Delta_{\xi}^2}{2} \left( \mathcal{K}(\xi_{j_1}, s_0) \mathcal{K}(\xi_{j_2}, t_0) + \mathcal{K}(\xi_{j_1}, s_0) \mathcal{K}(\xi_{j_2}, u_0) + \mathcal{K}(\xi_{j_1}, t_0) \mathcal{K}(\xi_{j_2}, u_0) \right) + (j_1 \leftrightarrow j_2),
\end{align*}
$$
and the partial waves become
$$
h_{\ell j} = h_{\ell j} f_0 + h_{\ell j; j_1} \sigma_{j_1} + h_{\ell j; j_1 j_2} \rho_{j_1 j_2}
$$
where the coefficients are
$$
\begin{align*}
    h_{\ell j} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \delta_{\ell 0}, \\
    h_{\ell j; j_1} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \left( \delta_{\ell 0} \mathcal{K}(\xi_{j_1}, s_j^+) + \Delta_{\xi} \hat{\mathbf{\Phi}}{_\ell}(s_j; \xi_{j_1}) \right), \\
    h_{\ell j; j_1 j_2} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \left( \Delta_{\xi} \mathcal{K}(\xi_{j_1}, s_j^+) \hat{\mathbf{\Phi}}_{\ell}(s_j; \xi_{j_2}) + \frac{\Delta_{\xi}^2}{2} \tilde{\mathbf{\Phi}}_{\ell}(s_j; \xi_{j_1}, \xi_{j_2}) \right) + (j_1 \leftrightarrow j_2),
\end{align*}
$$
where
$$
\mathcal{K}(\xi_{j_1}, s_j^+) = \hat{K}_{j j_1} + \frac{i}{2\pi} \delta_{j j_1}.
$$

The matrix $\hat{K}_{j_1 j_2}$ implements the principal part integral
$$
f(\xi) = \frac{1}{\pi} \int_0^\pi \frac{\sin(\xi_1)}{\cos(\xi) - \cos(\xi_1)} g(\xi_1) ~ \mathrm{d} \xi_1.
$$

To implement this integral, we extend $g(\xi_1)$ to the range $- \pi \leq \xi_1 \leq \pi$ by assuming that it is antisymmetric, *i.e.* $g(-\xi_1) = - g(\xi_1)$, so that the integrand is symmetric.

For the spectral densities $\sigma(\xi_1)$ and $\rho(\xi_1, \xi_2)$, this agrees with them being the imaginary part of the amplitude that changes sign across the cut.

Now, discretizing $f(\xi)$, we have
$$
f(\xi_j) = \sum_{j_1 = - M}^{M} \hat{K}_{j j_1} g(\xi_{j1}),
$$
where
$$
\hat{K}_{j_1 j_2} = \frac{1}{2M} \left( 1 - (-1)^{j_1 - j_2} \right) \cot \left( \frac{\pi}{2M} (j_1 - j_2) \right).
$$

### Rescaling

### Intermediate Points

### Full Subtractions

In [2]:
np.set_printoptions(precision=600)

########## VALUES ##########
# V = [[-1, 40, 20], [0, 40, 20], [4/3, 40, 20], [2, 40, 20], [3, 40, 20], [3.8, 40, 20]]
sa = [-1, 0, 4 / 3, 2, 3, 3.8][4]  # [?]
M = 40  # Number of interpolation points
lmax = 20  # Maximum angular momentum

# xi = np.pi/M * (np.arange(1, 2*M + 1) - 1/2)                      # Method 1
# xi = [np.pi/M * (j - 1/2) for j in range(1, 2*M+1)]               # Method 2
xi = np.linspace(
    np.pi / M * (1 / 2), np.pi / M * (2 * M - 1 / 2), 2 * M
)  # Interpolation points
s = sa + (4 - sa) / np.cos(xi / 2) ** 2  # Mapping from xi to s
rho = np.pi / 4 * np.sqrt((s - 4) / s)  # Mapping from s to rho

# UNUSED
# def zf(s):
#     return (np.sqrt(4 - sa) - np.sqrt(4 - s)) / (np.sqrt(4 - sa) + np.sqrt(4 - s))

print("sa =", sa)
print("M =", M)
print("lmax =", lmax)

# print("xi = ", xi)
# print("s = ", s) # algebraic type sa?
# print("rho = ", rho) # algebraic type sa?

sa = 3
M = 40
lmax = 20


We will now compute the real part of the kernel $\mathcal{K}$, which is $\hat{K}_{j_1 j_2}$, given by
$$
\hat{K}_{j_1 j_2} = \frac{1}{2M} \left( 1 - (-1)^{j_1 - j_2} \right) \cot \left( \frac{\pi}{2M} (j_1 - j_2) \right).
$$

In [3]:
def KHat(M, j1, j2):
    return (
        1 / (2 * M) * (1 - (-1) ** (j1 - j2)) * 1 / np.tan(np.pi / (2 * M) * (j1 - j2))
    )


KMatrix = np.zeros((2 * M, 2 * M))
print(np.shape(KMatrix))
for j1 in range(1, 2 * M + 1):
    for j2 in range(j1 + 1, 2 * M + 1):
        KMatrix[j1 - 1, j2 - 1] = KHat(M, j1, j2)
        KMatrix[j2 - 1, j1 - 1] = -KMatrix[j1 - 1, j2 - 1]

        np.nan_to_num(KMatrix, copy=False)

print(KMatrix)

(80, 80)
[[ 0.                  -0.636292489483927   -0.                  ...
   0.21122393349554025  0.                   0.6362924894839215 ]
 [ 0.636292489483927    0.                  -0.636292489483927   ...
   0.                   0.21122393349554025  0.                 ]
 [ 0.                   0.636292489483927    0.                  ...
   0.125683487303146    0.                   0.21122393349554025]
 ...
 [-0.21122393349554025 -0.                  -0.125683487303146   ...
   0.                  -0.636292489483927   -0.                 ]
 [-0.                  -0.21122393349554025 -0.                  ...
   0.636292489483927    0.                  -0.636292489483927  ]
 [-0.6362924894839215  -0.                  -0.21122393349554025 ...
   0.                   0.636292489483927    0.                 ]]


Now, we will compute the kernel in a different method by dividing the kernel $\mathcal{K}$ into its real and imaginary parts.

In [4]:
KMatrix_i = np.zeros((M, M))
KMatrix_d = np.zeros((M, M))
for j1 in range(1, M + 1):
    for j2 in range(1, M + 1):
        KMatrix_i[j1 - 1, j2 - 1] = -(
            KMatrix[j1 - 1, j2 - 1] - KMatrix[j1 - 1, 2 * M - j2]
        )
        KMatrix_d[j1 - 1, j2 - 1] = (
            KMatrix[j1 - 1, j2 - 1] + KMatrix[j1 - 1, 2 * M - j2]
        )

fvra = np.cos(4 * xi)
fvia = np.sin(4 * xi)
fviaN = np.dot(KMatrix, fvra)
norm1 = np.linalg.norm(fvia - fviaN)
print("norm1:", norm1)

fvr = np.cos(4 * xi)[:M]
fvi = np.sin(4 * xi)[:M]
fvrN = np.dot(KMatrix_i, fvi)
fviN = np.dot(KMatrix_d, fvr)
norm2 = np.linalg.norm(fvr - fvrN)
norm3 = np.linalg.norm(fvi - fviN)
print("norm2:", norm2)
print("norm3:", norm3)

norm1: 1.2855626395309555e-14
norm2: 4.974059352035939e-15
norm3: 7.461656892158854e-15


Now, we will take our initial values, defined by $s_0$, $t_0$, and $u_0$, such that
$$
    s_0 = t_0 = u_0 = \frac{4}{3}.
$$

In addition to that, we have
$$
    \mathcal{K} = \frac{1}{\pi} \frac{\sin(\xi)}{\frac{8}{s} - 1 - \cos(\xi)}
$$

$$
    \Kappa = \frac{1}{\pi} \sqrt{\frac{x_1 - 4}{4 - sa}} \frac{s - sa}{x_1 - s}
$$

$$
    \frac{\mathrm{d}}{\mathrm{d}s} \Kappa = \frac{1}{\pi} \sqrt{\frac{x_1 - 4}{4 - sa}} \frac{x_1 - sa}{(x_1 - s)^2}
$$

$$
    \frac{\mathrm{d}^2}{\mathrm{d}s^2} \Kappa = \frac{2}{\pi} \sqrt{\frac{x_1 - 4}{4 - sa}} \frac{x_1 - sa}{(x_1 - s)^3}
$$

In [ ]:
s0, t0, u0 = 4 / 3, 4 / 3, 4 / 3


def mathcalK(xi, s):
    return 1 / np.pi * np.sin(xi) / (8 / s - 1 - np.cos(xi))


def Kappa(sa, x1, s):
    return 1 / np.pi * np.sqrt((x1 - 4) / (4 - sa)) * (s - sa) / (x1 - s)


def Kappa1(sa, x1, s):
    return 1 / np.pi * np.sqrt((x1 - 4) / (4 - sa)) * (x1 - sa) / ((x1 - s) ** 2)


def Kappa2(sa, x1, s):
    return 2 / np.pi * np.sqrt((x1 - 4) / (4 - sa)) * (x1 - sa) / ((x1 - s) ** 3)


as1 = Kappa(sa, s, s0)
at1 = Kappa(sa, s, t0)
au1 = Kappa(sa, s, u0)

a0 = 1
a1 = as1 + at1 + au1
a2 = np.outer(as1, at1) + np.outer(as1, au1) + np.outer(at1, au1)
a2 = 1 / 2 * (a2 + a2.T)

bs1 = Kappa1(sa, s, s0)
bs2 = Kappa2(sa, s, s0)
b1 = 1 / 2 * bs2
b2 = 1 / 2 * (np.outer(as1, bs2) + np.outer(bs2, as1) - np.outer(bs1, bs1))

We will now define the Lambda function with the following argument.
$$
    \Lambda(\ell, s) \equiv \left( \frac{\sqrt{s} - 2}{\sqrt{s} + 2} \right)^{\left( \frac{\ell}{2} \right)}.
$$

In [6]:
def Lambda(l, s):
    return ((np.sqrt(s) - 2) / (np.sqrt(s) + 2)) ** (l / 2)


LambdaMatrix = np.ones((lmax + 1, M))
print(np.shape(LambdaMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        LambdaMatrix[l, j1 - 1] = Lambda(2 * l, s[j1 - 1])
        np.nan_to_num(LambdaMatrix, copy=False)

print(LambdaMatrix)

(21, 40)
[[1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00 1.0000000000000000e+00 1.0000000000000000e+00
  1.0000000000000000e+00]
 [2.4100746592356144e-05 2.172696113617302

We will now define the Legendre function of the second kind $Q_n$ with the following argument.
$$
    Q(\ell, x, s) \equiv Q_n \left( \ell, 1 + \frac{2x}{s - 4} \right).
$$

In [7]:
def LegendreQ(l, x, s):
    return lqn(l, 1 + 2 * x / (s - 4))


legendreQMatrix = np.zeros((lmax + 1, M, M))
print(np.shape(legendreQMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j in range(1, M + 1):
            legendreQMatrix[l, j1 - 1, j - 1] = LegendreQ(
                2 * (l + 1), s[j1 - 1], s[j - 1]
            )[0][2 * l]
            np.nan_to_num(legendreQMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j:", j)
            # print("leg:", LegendreQ(2*(l + 1), s[j1 - 1], s[j - 1])[0][2*l])

print(legendreQMatrix)

(21, 40, 40)
[[[4.8196846874129252e-005 4.3449735889606347e-004
   1.2109850078810221e-003 ... 1.6436266005667366e+000
   2.1443871012022555e+000 3.2378907663703433e+000]
  [4.8159622529139530e-005 4.3416190877461412e-004
   1.2100508017625752e-003 ... 1.6432547003547067e+000
   2.1440060667104275e+000 3.2375050239375249e+000]
  [4.8084886275372640e-005 4.3348841652384926e-004
   1.2081751675795747e-003 ... 1.6425071907789506e+000
   2.1432401764036091e+000 3.2367296595389954e+000]
  ...
  [1.8005468026385850e-006 1.6238071161974099e-005
   4.5290880632673529e-005 ... 3.3714707378698294e-001
   6.5214514532211387e-001 1.6137282547208671e+000]
  [6.6137667680653304e-007 5.9646212058454678e-006
   1.6636699636210353e-005 ... 1.5138350551619470e-001
   3.4313166293879549e-001 1.1460048337103146e+000]
  [7.4240930278020347e-008 6.6954449314640101e-007
   1.8675311933621706e-006 ... 1.9462362611557291e-002
   5.2500581406865317e-002 3.4618840534489526e-001]]

 [[1.4927759066135229e-014 1.09

With that definition, we can now rewrite $\hat{\mathbf{\Phi}}$ as
$$
    \hat{\mathbf{\Phi}} (\ell, x, s) = \frac{1}{\pi} \sqrt{\frac{x - 4}{4 - sa}} \left[ -2 \delta_{\ell 0} + \frac{4 (x - sa)}{s - 4} Q(\ell, x, s) \right].
$$

In [8]:
def PhiHat(l, x, s, sa, Q):
    # if (l == 0):
    return (
        1
        / np.pi
        * np.sqrt((x - 4) / (4 - sa))
        * ((l == 0) * (-2) + 4 * (x - sa) / (s - 4) * Q)
    )
    # else:
    #     return 1/np.pi * np.sqrt((x - 4)/(4 - sa)) * (4*(x - sa)/(s - 4) * Q)


PhiHatMatrix = np.zeros((lmax + 1, M, M))
print(np.shape(PhiHatMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j in range(1, M + 1):
            PhiHatMatrix[l, j1 - 1, j - 1] = PhiHat(
                l, s[j1 - 1], s[j - 1], sa, legendreQMatrix[l, j1 - 1, j - 1]
            )
            np.nan_to_num(PhiHatMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j:", j)
            # print("PhiHat:", PhiHat(l, s[j1 - 1], s[j - 1]) * legendreQMatrix[l, j1 - 1, j - 1])

print(PhiHatMatrix)

(21, 40, 40)
[[[-9.3754517965748373e-003 -9.3766592956776879e-003
   -9.3790855018014551e-003 ... -1.2102799344196752e-002
   -1.2315065686971780e-002 -1.2470374838667846e-002]
  [-2.8133567104259499e-002 -2.8137198924778017e-002
   -2.8144496284836455e-002 ... -3.6342348828214160e-002
   -3.6981601860462635e-002 -3.7449362031006211e-002]
  [-4.6913171170868011e-002 -4.6919255393136329e-002
   -4.6931480354964882e-002 ... -6.0684114610260562e-002
   -6.1757775617107664e-002 -6.2543530469784170e-002]
  ...
  [-1.8108978853048743e-001 -1.8118049361661370e-001
   -1.8136301794121359e-001 ... -2.0629885515870248e+000
   -3.4118834194577081e+000 -5.6263541448943561e+000]
  [-1.1109202768285256e-001 -1.1114868744913449e-001
   -1.1126270713016610e-001 ... -1.6469765286244393e+000
   -3.3610509617803936e+000 -8.0420394033888432e+000]
  [-3.7449460979612346e-002 -3.7468737597704693e-002
   -3.7507529769823113e-002 ... -6.6357264966419194e-001
   -1.7077316538674141e+000 -9.9640609913970835e+00

Additionally, we can rewrite $\tilde{\mathbf{\Phi}}$ as
$$
    \tilde{\mathbf{\Phi}} (\ell, x_1, x_2, s) = \frac{1}{\pi^2} \frac{\sqrt{(x_1 - 4)(x_2 - 4)}}{4 - sa} \left[ 2 \delta_{\ell 0} - \frac{4 (x_1 - sa) (s - 4 + sa + x_1)}{(s - 4 + x_1 + x_2) (s - 4)} Q(\ell, x_1, s) - \frac{4 (x_2 - sa) (s - 4 + sa + x_2)}{(s - 4 + x_1 + x_2) (s - 4)} (-1)^{\ell} Q(\ell, x_2, s) \right]
$$

In [9]:
def PhiTilde(l, x1, x2, s, sa, Q1, Q2):
    return (
        1
        / (np.pi) ** 2
        * np.sqrt((x1 - 4) * (x2 - 4))
        / (4 - sa)
        * (
            (l == 0) * 2
            - 4 * (x1 - sa) * (s - 4 + sa + x1) / (s - 4 + x1 + x2) / (s - 4) * Q1
            - 4
            * (x2 - sa)
            * (s - 4 + sa + x2)
            / (s - 4 + x1 + x2)
            / (s - 4)
            * (-1) ** l
            * Q2
        )
    )


PhiTildeMatrix = np.zeros((lmax + 1, M, M, M))
print(np.shape(PhiTildeMatrix))
for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                PhiTildeMatrix[l, j1 - 1, j2 - 1, j - 1] = PhiTilde(
                    l,
                    s[j1 - 1],
                    s[j2 - 1],
                    s[j - 1],
                    sa,
                    legendreQMatrix[l, j1 - 1, j - 1],
                    legendreQMatrix[l, j2 - 1, j - 1],
                )
                PhiTildeMatrix[l, j2 - 1, j1 - 1, j - 1] = PhiTildeMatrix[
                    l, j1 - 1, j2 - 1, j - 1
                ]
                np.nan_to_num(PhiTildeMatrix, copy=False)  # Replace NaN with 0

                # print("l:", l)
                # print("j:", j)
                # print("j1:", j1)
                # print("j2:", j2)
                # print("PhiTilde:", PhiTilde(l, xi[j1 - 1], xi[j2 - 1]))

# print(PhiTildeMatrix)

(21, 40, 40, 40)


We now convert the coefficients of the partial waves from a matrix form into a linear form to easily store in a data file.

In [10]:
Delta = np.pi / M


def delta(x, y):
    return 1 if x == y else 0


Mfn = int(M * (M + 1) / 2)
Mfl = (lmax + 1) * M  # Number of interpolation points in the partial waves
A0h = 1  # f0    -> functional c0
B0h = 0  # f0    -> functional c2
hl0_RM = np.zeros((Mfl))  # f0    -> Re(h)
A1h = np.zeros(M)  # sigma -> functional c0
B1h = np.zeros(M)  # sigma -> functional c2
hl1_RM = np.zeros((Mfl, M))  # sigma -> Re(h)
hl1_IM = np.zeros((Mfl, M))  # sigma -> Im(h)
A2h = np.zeros(Mfn)  # rho   -> functional c0
B2h = np.zeros(Mfn)  # rho   -> functional c2
hl2_RM = np.zeros((Mfl, Mfn))  # rho   -> Re(h)
hl2_IM = np.zeros((Mfl, Mfn))  # rho   -> Im(h)

print("Mfn:", Mfn)
print("Mfl:", Mfl)
print("Saving coefficients...")

# Now we put computation data in matrix form
##### F0 #####
for j in range(1, M + 1):
    jf1 = j - 1  # only l = 0
    hl0_RM[jf1] = 2 * rho[j - 1]

##### SIGMA #####
for j in range(1, M + 1):
    A1h[j - 1] = a1[j - 1] * Delta
    B1h[j - 1] = b1[j - 1] * Delta

for j in range(1, M + 1):
    hl1_IM[j - 1, j - 1] = 2 * rho[j - 1]

for j1 in range(1, M + 1):  # l = 0 part
    for j in range(1, M + 1):
        hl1_RM[j - 1, j1 - 1] = 2 * rho[j - 1] * KMatrix_i[j - 1, j1 - 1]

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            jf1 = l * M + j - 1
            jf2 = j1 - 1
            hl1_RM[jf1, jf2] += 2 * rho[j - 1] * Delta * PhiHatMatrix[l, j1 - 1, j - 1]

##### RHO #####
jf2 = 0
for j1 in range(1, M + 1):
    for j2 in range(j1, M + 1):
        A2h[jf2] = a2[j1 - 1, j2 - 1] * Delta**2
        B2h[jf2] = b2[j1 - 1, j2 - 1] * Delta**2
        jf2 += 1

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf2 = 0
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                jf1 = l * M + j - 1
                hl2_RM[jf1, jf2] = rho[j - 1] * (
                    KMatrix_i[j - 1, j1 - 1] * Delta * PhiHatMatrix[l, j2 - 1, j - 1]
                    + KMatrix_i[j - 1, j2 - 1] * Delta * PhiHatMatrix[l, j1 - 1, j - 1]
                    + Delta**2 * PhiTildeMatrix[l, j1 - 1, j2 - 1, j - 1]
                )
                jf2 += 1

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf2 = 0
        for j1 in range(1, M + 1):
            for j2 in range(j1, M + 1):
                jf1 = l * M + j - 1
                hl2_IM[jf1, jf2] = rho[j - 1] * (
                    delta(j - 1, j1 - 1) * Delta * PhiHatMatrix[l, j2 - 1, j - 1]
                    + delta(j - 1, j2 - 1) * Delta * PhiHatMatrix[l, j1 - 1, j - 1]
                )
                jf2 += 1

print("Done")

Mfn: 820
Mfl: 840
Saving coefficients...
Done


TBD

In [11]:
hl0_RMA = np.zeros(Mfl)  # f0    -> Re(h/Lambda)
hl1_IMA = np.zeros((Mfl, M))  # sigma -> Im(h/Lambda)
hl1_RMA = np.zeros((Mfl, M))  # sigma -> Re(h/Lambda)
hl2_IMA = np.zeros((Mfl, Mfn))  # rho   -> Im(h/Lambda)
hl2_RMA = np.zeros((Mfl, Mfn))  # rho   -> Re(h/Lambda)
hl1_IMB = np.zeros((Mfl, M))  # sigma -> Im(h/Lambda^2)
hl2_IMB = np.zeros((Mfl, Mfn))  # rho   -> Im(h/Lambda^2)


LambdaV = np.ones(Mfl)
for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        jf1 = l * M + j - 1
        LambdaV[jf1] = LambdaMatrix[l, j - 1]

for jf1 in range(1, Mfl + 1):
    hl0_RMA[jf1 - 1] = hl0_RM[jf1 - 1] / LambdaV[jf1 - 1]

for jf1 in range(1, Mfl + 1):
    for jf2 in range(0, M):
        hl1_RMA[jf1 - 1, jf2 - 1] = hl1_RM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl1_IMA[jf1 - 1, jf2 - 1] = hl1_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl1_IMB[jf1 - 1, jf2 - 1] = hl1_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1] ** 2

for jf1 in range(1, Mfl + 1):
    for jf2 in range(0, Mfn):
        hl2_RMA[jf1 - 1, jf2 - 1] = hl2_RM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl2_IMA[jf1 - 1, jf2 - 1] = hl2_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1]
        hl2_IMB[jf1 - 1, jf2 - 1] = hl2_IM[jf1 - 1, jf2 - 1] / LambdaV[jf1 - 1] ** 2

Save the data in a text file.

In [12]:
saf = str(sa).replace(".", "p")
dataKFile = os.path.join(
    os.getcwd(), f"primal_3.0_points_data_P2_nopole_sa{saf}_lmax{lmax}_M{M}.dat"
)

with open(dataKFile, "w") as fd:
    for j in range(1, M + 1):
        fd.write(f"{xi[j - 1]}\n")

    fd.write(f"{A0h}\n")

    fd.write(f"{B0h}\n")

    for jf1 in range(1, Mfl + 1):
        fd.write(f"{hl0_RM[jf1 - 1]}\n")

    for j in range(1, M + 1):
        fd.write(f"{A1h[j - 1]}\n")

    for j in range(1, M + 1):
        fd.write(f"{B1h[j - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_RM[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IM[jf1 - 1, jf2 - 1]}\n")

    for j in range(1, Mfn + 1):
        fd.write(f"{A2h[j - 1]}\n")

    for j in range(1, Mfn + 1):
        fd.write(f"{B2h[j - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_RM[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IM[jf1 - 1, jf2 - 1]}\n")

    ##### NOW RESCALED #####
    for jf1 in range(1, Mfl + 1):
        fd.write(f"{hl0_RMA[jf1 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_RMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_RMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IMA[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, M + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl1_IMB[jf1 - 1, jf2 - 1]}\n")

    for jf2 in range(1, Mfn + 1):
        for jf1 in range(1, Mfl + 1):
            fd.write(f"{hl2_IMB[jf1 - 1, jf2 - 1]}\n")

    for jf1 in range(1, Mfl + 1):
        fd.write(f"{LambdaV[jf1 - 1]}\n")

## Notes
A relation between the two solutions $P_n(x)$ and $Q_n(x)$ of the Legendre differential equation is given by
$$
Q_n(x) = P_n(x) \int \frac{1}{P_n^2(x) (1 - x^2)} ~ \mathrm{d} x.
$$